In [4]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV files
df1 = pd.read_csv('../data/original_paper_plot_infos.csv')
df2 = pd.read_csv('../data/rainbow_dqn_plot_infos.csv')
df_random = pd.read_csv('../data/random_plot_infos.csv')

label1 = 'DQN'
label2 = 'Rainbow DQN'
label_random = 'Random'

# Calculate the moving average with a window of 500
df1['total_reward_ma'] = df1['total_reward'].rolling(window=500).mean()
df1['total_loss_ma'] = df1['total_loss'].rolling(window=500).mean()
df1['total_q_values_ma'] = df1['total_q_values'].rolling(window=500).mean()

df2['total_reward_ma'] = df2['total_reward'].rolling(window=500).mean()
df2['total_loss_ma'] = df2['total_loss'].rolling(window=500).mean()
df2['total_q_values_ma'] = df2['total_q_values'].rolling(window=500).mean()

df_random['total_reward_ma'] = df_random['total_reward'].rolling(window=500).mean()

# Plot the data
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 10), dpi=200)

# Plot total_steps vs total_reward moving average
axes[0].plot(df1['total_steps'], df1['total_reward_ma'], label=label1 + ' Total Reward MA')
axes[0].plot(df2['total_steps'], df2['total_reward_ma'], label=label2 + ' Total Reward MA')
axes[0].plot(df_random['total_steps'], df_random['total_reward_ma'], label=label_random + ' Total Reward', linestyle='--')
axes[0].set_title('Total Steps vs Total Reward (MA)')
axes[0].set_xlabel('Total Steps (Millions)')
axes[0].set_ylabel('Total Reward (MA)')
axes[0].legend()
axes[0].grid(True)

# Plot total_steps vs total_loss moving average
axes[1].plot(df1['total_steps'], df1['total_loss_ma'], label=label1 + ' Total Loss MA')
axes[1].plot(df2['total_steps'], df2['total_loss_ma'], label=label2 + ' Total Loss MA')
axes[1].set_title('Total Steps vs Total Loss (MA)')
axes[1].set_xlabel('Total Steps (Millions)')
axes[1].set_ylabel('Total Loss (MA)')
axes[1].legend()
axes[1].grid(True)

# Plot total_steps vs total_q_values moving average
axes[2].plot(df1['total_steps'], df1['total_q_values_ma'], label=label1 + ' Total Q Values MA')
axes[2].plot(df2['total_steps'], df2['total_q_values_ma'], label=label2 + ' Total Q Values MA')
axes[2].set_title('Total Steps vs Total Q Values (MA)')
axes[2].set_xlabel('Total Steps (Millions)')
axes[2].set_ylabel('Total Q Values (MA)')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.show()